In [1]:
from torch_snippets import *

In [2]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('df.csv')
trn_df, val_df = train_test_split(df, random_state=10)

df_mini = df[df.ImageID.isin(df.ImageID.unique()[:500].tolist())]
trn_df_mini, val_df_mini = train_test_split(df_mini, random_state=10)

df_micro = df[df.ImageID.isin(df.ImageID.unique()[:50].tolist())]
trn_df_micro, val_df_micro = train_test_split(df_micro, random_state=10)

len(df)
df.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,...,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y
0,0000599864fd15b3,xclick,Bus,1,0.343750,0.908750,0.156162,0.650047,1,0,...,0,0,0.421875,0.343750,0.795000,0.908750,0.156162,0.512700,0.650047,0.457197
1,00006bdb1eb5cd74,xclick,Truck,1,0.276667,0.697500,0.141604,0.437343,1,0,...,0,0,0.299167,0.276667,0.697500,0.659167,0.141604,0.241855,0.352130,0.437343
2,00006bdb1eb5cd74,xclick,Truck,1,0.702500,0.999167,0.204261,0.409774,1,1,...,0,0,0.849167,0.702500,0.906667,0.999167,0.204261,0.398496,0.409774,0.295739
3,00010bf498b64bab,xclick,Bus,1,0.156250,0.371250,0.269188,0.705228,0,0,...,0,0,0.274375,0.371250,0.311875,0.156250,0.269188,0.493882,0.705228,0.521691
4,00013f14dd4e168f,xclick,Bus,1,0.287500,0.999375,0.194184,0.999062,0,1,...,0,0,0.920000,0.999375,0.648750,0.287500,0.194184,0.303940,0.999062,0.523452


In [28]:
labels = ['Bus', 'Truck']

def do(trn_df, val_df, folder):
    splits = [('train', trn_df), ('val', val_df)]
    for split, df in splits:
        os.makedirs(f'{folder}/labels', exist_ok=True)
        ImageIDs = df.ImageID.unique()
        for ImageID in ImageIDs:
            fname = ImageID
            _df = df[df['ImageID'] == ImageID][['LabelName','XMin','YMin','XMax','YMax']]
            _df['Xc'] = (_df['XMax'] + _df['XMin'])/2
            _df['Yc'] = (_df['YMax'] + _df['YMin'])/2
            _df['w'] = _df['XMax'] - _df['XMin']
            _df['h'] = _df['YMax'] - _df['YMin']
            _df['LabelName'] = _df['LabelName'].map(lambda x: labels.index(x))
            _df.drop(['XMin','YMin','XMax','YMax'], inplace=True, axis=1)
            with open(f'{folder}/labels/{fname}.txt', 'w') as f:
                for a,b,c,d,e in _df.values:
                    f.write(f'{int(a)} {b} {c} {d} {e}')
                    f.write('\n')
        with open(f'{folder}/{split}.txt', 'w') as f:
            for ImageID in ImageIDs:
                f.write(f'data/obj/{ImageID}.jpg\n')

do(trn_df_micro, val_df_micro, 'yolo_labels/micro/')
do(trn_df_mini, val_df_mini, 'yolo_labels/mini/')
do(trn_df, val_df, 'yolo_labels/all/')